# 尝试自动提取特征

In [1]:
import tensorflow as tf
import keras
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import os
import shutil
import sklearn
import cv2
import csv
import math
#import image
from skimage.io import imread
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import matplotlib.image as mpimg
#导入图表库
import matplotlib.pyplot as plt
from keras import models
from keras import layers
from keras import applications
from keras.layers import Dense
from keras.models import Sequential
from sklearn.preprocessing import normalize

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications.vgg19 import VGG19

from keras.applications.vgg19 import preprocess_input

from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing   

from sklearn.model_selection import KFold
from keras.utils import plot_model
# PCA 主成分分析
from sklearn.decomposition import PCA
from keras.callbacks import TensorBoard
print("hello tensorflow")

Using TensorFlow backend.


hello tensorflow


##  图像特征提取

### 拷贝 keras 内置经典网络

预训练的vgg-19

In [2]:
model_vgg19 = applications.VGG19(
    # 这里是利用预训练的模型来做特征提取
    # 因此我们不需要顶层的分类器网络部分的权重，只需要使用到训练好的卷积基。
    # 这也就是VGG19参数中include_top=False的含义
    include_top=False,
    weights='../../emotionData/vgg-19/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5'
)
model_vgg19.trainable = False

Instructions for updating:
Colocations handled automatically by placer.


## 网络构建

In [3]:
base_model = keras.Sequential()
model = VGG19(weights='imagenet', include_top=True)
print(model.summary())

base_model.add(model)
#model = keras.Sequential()
#model.add(model_vgg19)

output = keras.layers.Dense(512,activation="sigmoid")
base_model.add(output)

# 打印模型概况
print(base_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

### 特征提取

In [4]:
imagesDir = '../../emotionData/image/jpg/'

In [172]:
list = []
for num in range(1,501):
    img_path = imagesDir + str(num) + '.jpg'
    print(img_path)
    img = image.load_img(img_path, target_size=(224, 224))  # 加载图像，归一化大小
    x = image.img_to_array(img)   # 序列化
    x = np.expand_dims(x, axis=0)  # 展开
    x = preprocess_input(x)       # 预处理到0～1
    result = base_model.predict(x)  # 预测结果，500维的向量
    feature = np.array(result).tolist()
    list = list + feature
    #list.append(feature)
    
    

min_max_scaler = preprocessing.MinMaxScaler()
image_feature = pd.DataFrame(data=list)
image_feature_normed = min_max_scaler.fit_transform(image_feature)
image_feature_normed = pd.DataFrame(data=image_feature_normed.tolist())
image_feature_normed.to_csv('C:\\Users\\lab233\\Desktop\\emotionData\\image\\feature\\feature.csv')

../../emotionData/image/jpg/1.jpg
../../emotionData/image/jpg/2.jpg
../../emotionData/image/jpg/3.jpg
../../emotionData/image/jpg/4.jpg
../../emotionData/image/jpg/5.jpg
../../emotionData/image/jpg/6.jpg
../../emotionData/image/jpg/7.jpg
../../emotionData/image/jpg/8.jpg
../../emotionData/image/jpg/9.jpg
../../emotionData/image/jpg/10.jpg
../../emotionData/image/jpg/11.jpg
../../emotionData/image/jpg/12.jpg
../../emotionData/image/jpg/13.jpg
../../emotionData/image/jpg/14.jpg
../../emotionData/image/jpg/15.jpg
../../emotionData/image/jpg/16.jpg
../../emotionData/image/jpg/17.jpg
../../emotionData/image/jpg/18.jpg
../../emotionData/image/jpg/19.jpg
../../emotionData/image/jpg/20.jpg
../../emotionData/image/jpg/21.jpg
../../emotionData/image/jpg/22.jpg
../../emotionData/image/jpg/23.jpg
../../emotionData/image/jpg/24.jpg
../../emotionData/image/jpg/25.jpg
../../emotionData/image/jpg/26.jpg
../../emotionData/image/jpg/27.jpg
../../emotionData/image/jpg/28.jpg
../../emotionData/image/jpg/2

../../emotionData/image/jpg/232.jpg
../../emotionData/image/jpg/233.jpg
../../emotionData/image/jpg/234.jpg
../../emotionData/image/jpg/235.jpg
../../emotionData/image/jpg/236.jpg
../../emotionData/image/jpg/237.jpg
../../emotionData/image/jpg/238.jpg
../../emotionData/image/jpg/239.jpg
../../emotionData/image/jpg/240.jpg
../../emotionData/image/jpg/241.jpg
../../emotionData/image/jpg/242.jpg
../../emotionData/image/jpg/243.jpg
../../emotionData/image/jpg/244.jpg
../../emotionData/image/jpg/245.jpg
../../emotionData/image/jpg/246.jpg
../../emotionData/image/jpg/247.jpg
../../emotionData/image/jpg/248.jpg
../../emotionData/image/jpg/249.jpg
../../emotionData/image/jpg/250.jpg
../../emotionData/image/jpg/251.jpg
../../emotionData/image/jpg/252.jpg
../../emotionData/image/jpg/253.jpg
../../emotionData/image/jpg/254.jpg
../../emotionData/image/jpg/255.jpg
../../emotionData/image/jpg/256.jpg
../../emotionData/image/jpg/257.jpg
../../emotionData/image/jpg/258.jpg
../../emotionData/image/jpg/

../../emotionData/image/jpg/460.jpg
../../emotionData/image/jpg/461.jpg
../../emotionData/image/jpg/462.jpg
../../emotionData/image/jpg/463.jpg
../../emotionData/image/jpg/464.jpg
../../emotionData/image/jpg/465.jpg
../../emotionData/image/jpg/466.jpg
../../emotionData/image/jpg/467.jpg
../../emotionData/image/jpg/468.jpg
../../emotionData/image/jpg/469.jpg
../../emotionData/image/jpg/470.jpg
../../emotionData/image/jpg/471.jpg
../../emotionData/image/jpg/472.jpg
../../emotionData/image/jpg/473.jpg
../../emotionData/image/jpg/474.jpg
../../emotionData/image/jpg/475.jpg
../../emotionData/image/jpg/476.jpg
../../emotionData/image/jpg/477.jpg
../../emotionData/image/jpg/478.jpg
../../emotionData/image/jpg/479.jpg
../../emotionData/image/jpg/480.jpg
../../emotionData/image/jpg/481.jpg
../../emotionData/image/jpg/482.jpg
../../emotionData/image/jpg/483.jpg
../../emotionData/image/jpg/484.jpg
../../emotionData/image/jpg/485.jpg
../../emotionData/image/jpg/486.jpg
../../emotionData/image/jpg/

## 音频特征提取

### MFCC & Chroma Frequencies 提取

In [180]:
def mtrix2vector(list):
    returnVect = np.zeros((1, 400))
    for i in range(20):
        lineStr = list[i]
        for j in range(20):
            returnVect[0, 20*i+j] = int(lineStr[j])
    return returnVect.tolist()

In [277]:
import librosa
import librosa.display
import librosa.feature
from numpy import *


def extracteMFCC(dir):
    list = []
    del list[:]
    for i in range(0,100):
        if os.path.exists(dir + str(i+1) + '.wav') == False:
            print(dir + str(i+1) + '.wav: 文件不存在！')
            arr = zeros([1,512])
            feature = arr.tolist()
            list = list + feature
            continue
        x, sr = librosa.load(dir + str(i+1) + '.wav')
        print(dir + str(i+1) + '.wav')
        # MFCC
        mfccs = librosa.feature.mfcc(x, sr = sr)
        mfccs = sklearn.preprocessing.scale(mfccs, axis=1)
        # 使用主成分降维
        pca_3 = PCA(n_components=20)
        data_pca_3 = pca_3.fit_transform(mfccs)
        mfcc_feature = mtrix2vector(data_pca_3)
        # Spectral Rolloff：频谱滚降点
        spectral_rolloff = librosa.feature.spectral_rolloff(x+0.01, sr=sr)[0]
        spectral_rolloff = spectral_rolloff[:112]
        rolloff_feature = mat(spectral_rolloff)
        #feature = mfcc_feature + rolloff_feature
        feature = np.hstack((mfcc_feature,rolloff_feature))
        print(feature.shape)
        feature = feature.tolist()
        list = list + feature
        #print(np.array(list).shape)
        #print(list)
        #list.shape
    return list

In [278]:
audioDir_1 = '../../emotionData/audio/1st100song/'
audioDir_2 = '../../emotionData/audio/2 100songs/'
audioDir_3 = '../../emotionData/audio/3 100songs/'
audioDir_4 = '../../emotionData/audio/4 100songs/'
audioDir_5 = '../../emotionData/audio/5 100songs/'
list1 = extracteMFCC(audioDir_1)
list2 = extracteMFCC(audioDir_2)
list3 = extracteMFCC(audioDir_3)
list4 = extracteMFCC(audioDir_4)
list5 = extracteMFCC(audioDir_5)
list = list1
list = list1 + list2 + list3 + list4 + list5
audio_feature = pd.DataFrame(data=list)

../../emotionData/audio/1st100song/1.wav
(1, 512)
../../emotionData/audio/1st100song/2.wav
(1, 512)
../../emotionData/audio/1st100song/3.wav
(1, 512)
../../emotionData/audio/1st100song/4.wav
(1, 512)
../../emotionData/audio/1st100song/5.wav
(1, 512)
../../emotionData/audio/1st100song/6.wav
(1, 512)
../../emotionData/audio/1st100song/7.wav
(1, 512)
../../emotionData/audio/1st100song/8.wav
(1, 512)
../../emotionData/audio/1st100song/9.wav
(1, 512)
../../emotionData/audio/1st100song/10.wav
(1, 512)
../../emotionData/audio/1st100song/11.wav
(1, 512)
../../emotionData/audio/1st100song/12.wav
(1, 512)
../../emotionData/audio/1st100song/13.wav
(1, 512)
../../emotionData/audio/1st100song/14.wav
(1, 512)
../../emotionData/audio/1st100song/15.wav
(1, 512)
../../emotionData/audio/1st100song/16.wav
(1, 512)
../../emotionData/audio/1st100song/17.wav
(1, 512)
../../emotionData/audio/1st100song/18.wav
(1, 512)
../../emotionData/audio/1st100song/19.wav
(1, 512)
../../emotionData/audio/1st100song/20.wa

../../emotionData/audio/2 100songs/63.wav
(1, 512)
../../emotionData/audio/2 100songs/64.wav
(1, 512)
../../emotionData/audio/2 100songs/65.wav
(1, 512)
../../emotionData/audio/2 100songs/66.wav
(1, 512)
../../emotionData/audio/2 100songs/67.wav
(1, 512)
../../emotionData/audio/2 100songs/68.wav
(1, 512)
../../emotionData/audio/2 100songs/69.wav
(1, 512)
../../emotionData/audio/2 100songs/70.wav
(1, 512)
../../emotionData/audio/2 100songs/71.wav
(1, 512)
../../emotionData/audio/2 100songs/72.wav
(1, 512)
../../emotionData/audio/2 100songs/73.wav
(1, 512)
../../emotionData/audio/2 100songs/74.wav
(1, 512)
../../emotionData/audio/2 100songs/75.wav
(1, 512)
../../emotionData/audio/2 100songs/76.wav
(1, 512)
../../emotionData/audio/2 100songs/77.wav
(1, 512)
../../emotionData/audio/2 100songs/78.wav
(1, 512)
../../emotionData/audio/2 100songs/79.wav
(1, 512)
../../emotionData/audio/2 100songs/80.wav
(1, 512)
../../emotionData/audio/2 100songs/81.wav
(1, 512)
../../emotionData/audio/2 100so

../../emotionData/audio/4 100songs/24.wav
(1, 512)
../../emotionData/audio/4 100songs/25.wav
(1, 512)
../../emotionData/audio/4 100songs/26.wav
(1, 512)
../../emotionData/audio/4 100songs/27.wav
(1, 512)
../../emotionData/audio/4 100songs/28.wav
(1, 512)
../../emotionData/audio/4 100songs/29.wav
(1, 512)
../../emotionData/audio/4 100songs/30.wav
(1, 512)
../../emotionData/audio/4 100songs/31.wav
(1, 512)
../../emotionData/audio/4 100songs/32.wav
(1, 512)
../../emotionData/audio/4 100songs/33.wav
(1, 512)
../../emotionData/audio/4 100songs/34.wav
(1, 512)
../../emotionData/audio/4 100songs/35.wav
(1, 512)
../../emotionData/audio/4 100songs/36.wav
(1, 512)
../../emotionData/audio/4 100songs/37.wav
(1, 512)
../../emotionData/audio/4 100songs/38.wav
(1, 512)
../../emotionData/audio/4 100songs/39.wav
(1, 512)
../../emotionData/audio/4 100songs/40.wav
(1, 512)
../../emotionData/audio/4 100songs/41.wav
(1, 512)
../../emotionData/audio/4 100songs/42.wav
(1, 512)
../../emotionData/audio/4 100so

../../emotionData/audio/5 100songs/85.wav
(1, 512)
../../emotionData/audio/5 100songs/86.wav
(1, 512)
../../emotionData/audio/5 100songs/87.wav
(1, 512)
../../emotionData/audio/5 100songs/88.wav
(1, 512)
../../emotionData/audio/5 100songs/89.wav
(1, 512)
../../emotionData/audio/5 100songs/90.wav
(1, 512)
../../emotionData/audio/5 100songs/91.wav
(1, 512)
../../emotionData/audio/5 100songs/92.wav
(1, 512)
../../emotionData/audio/5 100songs/93.wav
(1, 512)
../../emotionData/audio/5 100songs/94.wav
(1, 512)
../../emotionData/audio/5 100songs/95.wav
(1, 512)
../../emotionData/audio/5 100songs/96.wav
(1, 512)
../../emotionData/audio/5 100songs/97.wav
(1, 512)
../../emotionData/audio/5 100songs/98.wav
(1, 512)
../../emotionData/audio/5 100songs/99.wav
(1, 512)
../../emotionData/audio/5 100songs/100.wav
(1, 512)


In [279]:

min_max_scaler = preprocessing.MinMaxScaler()  
audio_feature_normed = min_max_scaler.fit_transform(audio_feature).tolist()
audio_feature_normed = pd.DataFrame(data=audio_feature_normed)
audio_feature_normed.to_csv('C:\\Users\\lab233\\Desktop\\emotionData\\audio\\feature\\feature.csv')

## sound net 提取音频特征

In [11]:
os.listdir("../../SoundNet-tensorflow/output/")
feature = np.load('../../SoundNet-tensorflow/output/tf_fea08.npy')
print(feature)
print(feature.shape)

[[0.         1.133764   0.35062057 ... 1.2682095  0.42631838 0.        ]
 [0.         0.37141797 0.3779703  ... 0.7573055  0.41166326 0.        ]
 [0.         0.37159377 0.3780061  ... 0.7574363  0.4115947  0.        ]
 ...
 [0.         0.01315236 0.         ... 0.5299513  0.6857818  0.6103643 ]
 [0.         0.37279242 0.00335231 ... 1.1156482  0.         0.49528283]
 [0.         0.9434061  0.2554638  ... 1.0339558  0.698925   0.40814018]]
(5198, 32)


In [15]:
str1 = "aaa" + "bbb"
print(str1)

aaabbb


## 数据拼接 & 保存

In [2]:
label_data = pd.read_csv(open("../../emotionData/pearson.csv",encoding='utf-8'))
music_data = pd.read_csv(open("../../emotionData/audio/feature/feature.csv",encoding='utf-8'))
image_data = pd.read_csv(open("../../emotionData/image/feature/feature.csv",encoding='utf-8'))
print(label_data.shape)
print(music_data.shape)
print(image_data.shape)

(250000, 4)
(500, 512)
(500, 512)


In [3]:
label_data_frame = label_data.values
music_data_mat = music_data.values
image_data_mat = image_data.values

def processData(image_num, audio_num, pearson):
    list = []
    a = image_data_mat[image_num-1].tolist()
    b = music_data_mat[audio_num-1].tolist()
    list = a + b
    dataMat = mat(list)
    print(dataMat.shape)
    return list
    

def wirteFinalDataToCSV_1():
    list = []
    flag = True
    for i in range(249990,250000):
        str_x = label_data_frame[i]
        print(i+1)
        print("image: "+str_x[:1][0][1:]+"    audio: "+str_x[1:2][0][1:]+"   pearson: "+str(str_x[3:4][0]))
        if flag == True:
            list = np.mat(processData(int(str_x[:1][0][1:]),int(str_x[1:2][0][1:]),str_x[3:4][0]))
            flag = False
        else:
            list = np.vstack((list,processData(int(str_x[:1][0][1:]),int(str_x[1:2][0][1:]),str_x[3:4][0])))
        print()
    return list
    
list_1 = wirteFinalDataToCSV_1()
dataFrame = pd.DataFrame(list_1)
print(dataFrame.shape)
dataFrame.to_csv('C:\\Users\\lab233\\Desktop\\emotionData\\finalData_2.csv')

249991
image: 364    audio: 97   pearson: 0.999967


NameError: name 'mat' is not defined

## 训练模型

#### 定义模型

In [3]:
# from keras.applications.resnet50 import ResNet50


# # pretrained model is of 3 channels
# def create_model_resnet50():
#     #print(RESNET_VER)
#     base_model =ResNet50(weights=None, include_top=False)
#     base_model.load_weights('../../emotionData/ResNet/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')
    
#     x0 = base_model.output
#     x1 = GlobalAveragePooling2D()(x0)
#     x2 = GlobalMaxPooling2D()(x0)
#     x = Concatenate()([x1,x2])
    
#     x = BatchNormalization()(x)
# #     x = Dropout(0.5)(x)
#     x = Dense(1024, activation='relu')(x)
#     x = BatchNormalization()(x)
#     x = Dropout(0.5)(x)
#     x = Dense(512, activation='relu')(x)
#     x = BatchNormalization()(x)
#     x = Dropout(0.5)(x)

#     predictions = Dense(1, activation='sigmoid')(x)

#     # this is the model we will train
#     model = Model(inputs=base_model.input, outputs=predictions)
#     return model


In [2]:
# model = create_model_resnet50()

# for layer in model.layers:
#     layer.trainable = False

# for i in range(-10,0):
#     model.layers[i].trainable = True

# model.compile(
#     loss='mean_squared_error',
#     optimizer='adam', 
#     metrics=['mse']
# )

# model.summary()

### callbacks

In [5]:
# create callbacks list
from keras.callbacks import (ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, CSVLogger)
                             
from sklearn.model_selection import train_test_split

LR_FACTOR = 0.2
PATIENCE = 2
checkpoint_file = './working/resnet50_focal.h5'

checkpoint = ModelCheckpoint(
    checkpoint_file, 
    monitor='val_loss', 
    verbose=1, 
    save_best_only=True, 
    mode='min', 
    save_weights_only = False
)

reduceLROnPlat = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=LR_FACTOR, 
    patience=PATIENCE,
    verbose=1, 
    mode='auto', 
    min_delta=0.0001, 
    cooldown=0, 
    min_lr=1e-5
)

csv_logger = CSVLogger(
    filename='./working/training_log.csv',
    separator=',',
    append=True
)

### 分割 & 准备数据

In [36]:
# 读取数据
X = pd.read_csv(open("../../emotionData/finalData.csv",encoding='utf-8'))
#X = pd.DataFrame(X)
#
X = X.values
print(X.shape)

(250000, 1024)


In [37]:
with open('../../emotionData/pearson.CSV',"rt", encoding="utf-8") as vsvfile:
    reader = csv.reader(vsvfile)
    y = [row[3] for row in reader]
#y = pd.DataFrame(y)
y = y[1:]
#
y = pd.DataFrame(y)
print(y.shape)

(250000, 1)


In [38]:
from keras.layers.normalization import BatchNormalization

model = Sequential()

model.add(Dense(1024, input_dim=1024, activation='relu'))
model.add(keras.layers.normalization.BatchNormalization(axis=-1, 
                                              momentum=0.99, 
                                              epsilon=0.001, 
                                              center=True, 
                                              scale=True, 
                                              beta_initializer='zeros', 
                                              gamma_initializer='ones',
                                              ))

model.add(Dense(1024, input_dim=1024, activation='relu'))
model.add(keras.layers.normalization.BatchNormalization(axis=-1, 
                                              momentum=0.99, 
                                              epsilon=0.001, 
                                              center=True, 
                                              scale=True, 
                                              beta_initializer='zeros', 
                                              gamma_initializer='ones',
                                              ))


model.add(Dense(512, input_dim=1024, activation='relu'))
model.add(keras.layers.normalization.BatchNormalization(axis=-1, 
                                              momentum=0.99, 
                                              epsilon=0.001, 
                                              center=True, 
                                              scale=True, 
                                              beta_initializer='zeros', 
                                              gamma_initializer='ones',
                                              ))

model.add(Dense(512, input_dim=512, activation='relu'))
model.add(keras.layers.normalization.BatchNormalization(axis=-1, 
                                              momentum=0.99, 
                                              epsilon=0.001, 
                                              center=True, 
                                              scale=True, 
                                              beta_initializer='zeros', 
                                              gamma_initializer='ones',
                                              ))

model.add(Dense(256, input_dim=512, activation='relu'))
model.add(keras.layers.normalization.BatchNormalization(axis=-1, 
                                              momentum=0.99, 
                                              epsilon=0.001, 
                                              center=True, 
                                              scale=True, 
                                              beta_initializer='zeros', 
                                              gamma_initializer='ones',
                                              ))

model.add(Dense(256, input_dim=256, activation='relu'))
model.add(keras.layers.normalization.BatchNormalization(axis=-1, 
                                              momentum=0.99, 
                                              epsilon=0.001, 
                                              center=True, 
                                              scale=True, 
                                              beta_initializer='zeros', 
                                              gamma_initializer='ones',
                                              ))

model.add(Dense(128, input_dim=256, activation='relu'))
model.add(keras.layers.normalization.BatchNormalization(axis=-1, 
                                              momentum=0.99, 
                                              epsilon=0.001, 
                                              center=True, 
                                              scale=True, 
                                              beta_initializer='zeros', 
                                              gamma_initializer='ones',
                                              ))

model.add(Dense(128, input_dim=128, activation='relu'))
model.add(keras.layers.normalization.BatchNormalization(axis=-1, 
                                              momentum=0.99, 
                                              epsilon=0.001, 
                                              center=True, 
                                              scale=True, 
                                              beta_initializer='zeros', 
                                              gamma_initializer='ones',
                                              ))

model.add(Dense(64, activation='relu'))
model.add(keras.layers.normalization.BatchNormalization(axis=-1, 
                                              momentum=0.99, 
                                              epsilon=0.001, 
                                              center=True, 
                                              scale=True, 
                                              beta_initializer='zeros', 
                                              gamma_initializer='ones',
                                              ))

model.add(Dense(64, activation='relu'))
model.add(keras.layers.normalization.BatchNormalization(axis=-1, 
                                              momentum=0.99, 
                                              epsilon=0.001, 
                                              center=True, 
                                              scale=True, 
                                              beta_initializer='zeros', 
                                              gamma_initializer='ones',
                                              ))

model.add(Dense(32, activation='relu'))
model.add(keras.layers.normalization.BatchNormalization(axis=-1, 
                                              momentum=0.99, 
                                              epsilon=0.001, 
                                              center=True, 
                                              scale=True, 
                                              beta_initializer='zeros', 
                                              gamma_initializer='ones',
                                              ))

model.add(Dense(16, activation='relu'))
model.add(keras.layers.normalization.BatchNormalization(axis=-1, 
                                              momentum=0.99, 
                                              epsilon=0.001, 
                                              center=True, 
                                              scale=True, 
                                              beta_initializer='zeros', 
                                              gamma_initializer='ones',
                                              ))

model.add(Dense(16, activation='relu'))
model.add(keras.layers.normalization.BatchNormalization(axis=-1, 
                                              momentum=0.99, 
                                              epsilon=0.001, 
                                              center=True, 
                                              scale=True, 
                                              beta_initializer='zeros', 
                                              gamma_initializer='ones',
                                              ))

model.add(Dense(8, activation='relu'))
model.add(keras.layers.normalization.BatchNormalization(axis=-1, 
                                              momentum=0.99, 
                                              epsilon=0.001, 
                                              center=True, 
                                              scale=True, 
                                              beta_initializer='zeros', 
                                              gamma_initializer='ones',
                                              ))

model.add(Dense(8, activation='relu'))
model.add(keras.layers.normalization.BatchNormalization(axis=-1, 
                                              momentum=0.99, 
                                              epsilon=0.001, 
                                              center=True, 
                                              scale=True, 
                                              beta_initializer='zeros', 
                                              gamma_initializer='ones',
                                              ))


model.add(Dense(1, activation='sigmoid'))


In [39]:
from keras import backend as K

from sklearn import metrics

def mre(y_true, y_pred):
    return K.mean(K.sum(K.abs(100*K.ones_like(y_true) * (y_true - y_pred) / y_true)))


def pearson_r(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x, axis=0)
    my = K.mean(y, axis=0)
    xm, ym = x - mx, y - my
    r_num = K.sum(xm * ym)
    x_square_sum = K.sum(xm * xm)
    y_square_sum = K.sum(ym * ym)
    r_den = K.sqrt(x_square_sum * y_square_sum)
    r = r_num / r_den
    return K.mean(r)

def r_square_final(y_true, y_pred):
    SSR = K.mean(K.square(y_pred-K.mean(y_true)),axis=-1)
    SST = K.mean(K.square(y_true-K.mean(y_true)),axis=-1)
    return SSR/SST





In [40]:

model.compile(
    loss='mean_squared_error', 
    optimizer='adam',
    metrics=['mean_squared_error',
             'mean_absolute_error',
             'mean_absolute_percentage_error',
             'mean_squared_logarithmic_error',
             r_square_final,
            # metrics.r2_score,
            # metrics.r2_score,
             pearson_r,
             mre]
)

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
batch_normalization_16 (Batc (None, 1024)              4096      
_________________________________________________________________
dense_18 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
batch_normalization_17 (Batc (None, 1024)              4096      
_________________________________________________________________
dense_19 (Dense)             (None, 512)               524800    
_________________________________________________________________
batch_normalization_18 (Batc (None, 512)               2048      
_________________________________________________________________
dense_20 (Dense)             (None, 512)               262656    
__________

In [41]:
callbacks_list = [
    # 训练可视化
    keras.callbacks.TensorBoard(
        log_dir='./logs',  # log 目录
        histogram_freq=0,  # 按照何等频率（epoch）来计算直方图，0为不计算
        #batch_size=32,     # 用多大量的数据计算直方图
        write_graph=True,  # 是否存储网络结构图
        write_grads=True, # 是否可视化梯度直方图
        write_images=True,# 是否可视化参数
        embeddings_freq=0,
        embeddings_layer_names=None, 
        embeddings_metadata=None),
    
    # 训练自动更改学习率
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_mean_squared_error', 
        factor=0.2,
        patience=5,
        verbose=1,
        mode='min',
        min_delta=0.0001, 
        cooldown=0, 
        min_lr=0.0001),
    
    # 当被监测的数量不再提升，则停止训练
    keras.callbacks.EarlyStopping(
        monitor='val_mean_squared_error', 
        min_delta=0, 
        patience=30, 
        verbose=1, 
        mode='min', 
        baseline=None, 
        restore_best_weights=False)

]



### 提高gpu占用率

In [42]:
import keras.backend.tensorflow_backend as KTF #进行配置，每个GPU使用80%上限现存


#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # 使用编号为0号的GPU

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8 # 每个GPU现存上届控制在80%以内
session = tf.Session(config=config) # 设置session KTF.set_session(session )
#设置session
KTF.set_session(session )

#注意：
#config.gpu_options.per_process_gpu_memory_fraction = 0.8 # 每个GPU现存上届控制在80%以内
#设置GPU使用上限，可以有效避免显存不足而导致的jupyter 问题:
#The kernel appears to have died. It will restart automatically

#### 训练

In [43]:
keras.backend.get_session().run(tf.global_variables_initializer())


his=model.fit(
    X,
    y,
    
    validation_split=0.1,
    epochs=120,
    batch_size=1024,
    
    shuffle=True,
    verbose=1,
    
    
    callbacks=callbacks_list
)

Train on 225000 samples, validate on 25000 samples
Epoch 1/120
225000/225000 [==============================] - 54s 241us/step - loss: 0.0551 - mean_squared_error: 0.0551 - mean_absolute_error: 0.1988 - mean_absolute_percentage_error: 2115.6785 - mean_squared_logarithmic_error: 0.0264 - r_square_final: 365248.9864 - pearson_r: 0.2459 - mre: inf - val_loss: 0.2046 - val_mean_squared_error: 0.2046 - val_mean_absolute_error: 0.4349 - val_mean_absolute_percentage_error: 46.0924 - val_mean_squared_logarithmic_error: 0.0723 - val_r_square_final: inf - val_pearson_r: -0.0060 - val_mre: 46646.8507
Epoch 2/120
225000/225000 [==============================] - 52s 230us/step - loss: 0.0378 - mean_squared_error: 0.0378 - mean_absolute_error: 0.1556 - mean_absolute_percentage_error: 2064.7045 - mean_squared_logarithmic_error: 0.0181 - r_square_final: 17765.9436 - pearson_r: 0.5926 - mre: inf - val_loss: 0.1496 - val_mean_squared_error: 0.1496 - val_mean_absolute_error: 0.3331 - val_mean_absolute_pe

Epoch 16/120
225000/225000 [==============================] - 52s 230us/step - loss: 0.0082 - mean_squared_error: 0.0082 - mean_absolute_error: 0.0589 - mean_absolute_percentage_error: 354.7805 - mean_squared_logarithmic_error: 0.0040 - r_square_final: 69761.9228 - pearson_r: 0.9278 - mre: inf - val_loss: 0.0503 - val_mean_squared_error: 0.0503 - val_mean_absolute_error: 0.1488 - val_mean_absolute_percentage_error: 15.6973 - val_mean_squared_logarithmic_error: 0.0186 - val_r_square_final: inf - val_pearson_r: 0.0019 - val_mre: 15883.5484
Epoch 17/120
225000/225000 [==============================] - 52s 229us/step - loss: 0.0081 - mean_squared_error: 0.0081 - mean_absolute_error: 0.0583 - mean_absolute_percentage_error: 309.3762 - mean_squared_logarithmic_error: 0.0039 - r_square_final: 24424.9368 - pearson_r: 0.9288 - mre: inf - val_loss: 0.0505 - val_mean_squared_error: 0.0505 - val_mean_absolute_error: 0.1456 - val_mean_absolute_percentage_error: 15.3594 - val_mean_squared_logarithmi

Epoch 31/120
225000/225000 [==============================] - 51s 228us/step - loss: 0.0047 - mean_squared_error: 0.0047 - mean_absolute_error: 0.0370 - mean_absolute_percentage_error: 146.8571 - mean_squared_logarithmic_error: 0.0023 - r_square_final: 524.1508 - pearson_r: 0.9594 - mre: inf - val_loss: 0.0410 - val_mean_squared_error: 0.0410 - val_mean_absolute_error: 0.1211 - val_mean_absolute_percentage_error: 12.7664 - val_mean_squared_logarithmic_error: 0.0157 - val_r_square_final: inf - val_pearson_r: 0.0057 - val_mre: 12935.5404
Epoch 32/120
225000/225000 [==============================] - 51s 228us/step - loss: 0.0047 - mean_squared_error: 0.0047 - mean_absolute_error: 0.0366 - mean_absolute_percentage_error: 147.9328 - mean_squared_logarithmic_error: 0.0023 - r_square_final: 2960.3856 - pearson_r: 0.9598 - mre: inf - val_loss: 0.0406 - val_mean_squared_error: 0.0406 - val_mean_absolute_error: 0.1144 - val_mean_absolute_percentage_error: 12.0613 - val_mean_squared_logarithmic_e

Epoch 46/120
225000/225000 [==============================] - 51s 228us/step - loss: 0.0043 - mean_squared_error: 0.0043 - mean_absolute_error: 0.0323 - mean_absolute_percentage_error: 136.4908 - mean_squared_logarithmic_error: 0.0021 - r_square_final: 161015.2658 - pearson_r: 0.9634 - mre: inf - val_loss: 0.0409 - val_mean_squared_error: 0.0409 - val_mean_absolute_error: 0.1147 - val_mean_absolute_percentage_error: 12.0923 - val_mean_squared_logarithmic_error: 0.0160 - val_r_square_final: inf - val_pearson_r: 0.0070 - val_mre: 12257.7705
Epoch 47/120
225000/225000 [==============================] - 51s 227us/step - loss: 0.0043 - mean_squared_error: 0.0043 - mean_absolute_error: 0.0324 - mean_absolute_percentage_error: 153.5946 - mean_squared_logarithmic_error: 0.0021 - r_square_final: 1943.9595 - pearson_r: 0.9634 - mre: inf - val_loss: 0.0388 - val_mean_squared_error: 0.0388 - val_mean_absolute_error: 0.1103 - val_mean_absolute_percentage_error: 11.6342 - val_mean_squared_logarithmi

225000/225000 [==============================] - 52s 230us/step - loss: 0.0041 - mean_squared_error: 0.0041 - mean_absolute_error: 0.0307 - mean_absolute_percentage_error: 146.4425 - mean_squared_logarithmic_error: 0.0020 - r_square_final: 1701.1248 - pearson_r: 0.9648 - mre: inf - val_loss: 0.0408 - val_mean_squared_error: 0.0408 - val_mean_absolute_error: 0.1124 - val_mean_absolute_percentage_error: 11.8495 - val_mean_squared_logarithmic_error: 0.0161 - val_r_square_final: inf - val_pearson_r: 0.0083 - val_mre: 12014.9094
Epoch 62/120
225000/225000 [==============================] - 52s 229us/step - loss: 0.0041 - mean_squared_error: 0.0041 - mean_absolute_error: 0.0304 - mean_absolute_percentage_error: 174.6411 - mean_squared_logarithmic_error: 0.0020 - r_square_final: 19603.1127 - pearson_r: 0.9649 - mre: inf - val_loss: 0.0414 - val_mean_squared_error: 0.0414 - val_mean_absolute_error: 0.1113 - val_mean_absolute_percentage_error: 11.7336 - val_mean_squared_logarithmic_error: 0.016

225000/225000 [==============================] - 52s 229us/step - loss: 0.0040 - mean_squared_error: 0.0040 - mean_absolute_error: 0.0296 - mean_absolute_percentage_error: 138.7742 - mean_squared_logarithmic_error: 0.0019 - r_square_final: 1047.6546 - pearson_r: 0.9657 - mre: inf - val_loss: 0.0404 - val_mean_squared_error: 0.0404 - val_mean_absolute_error: 0.1117 - val_mean_absolute_percentage_error: 11.7850 - val_mean_squared_logarithmic_error: 0.0159 - val_r_square_final: inf - val_pearson_r: 0.0095 - val_mre: 11949.4214
Epoch 77/120
225000/225000 [==============================] - 52s 229us/step - loss: 0.0040 - mean_squared_error: 0.0040 - mean_absolute_error: 0.0291 - mean_absolute_percentage_error: 141.5591 - mean_squared_logarithmic_error: 0.0019 - r_square_final: 35257.9623 - pearson_r: 0.9658 - mre: inf - val_loss: 0.0406 - val_mean_squared_error: 0.0406 - val_mean_absolute_error: 0.1098 - val_mean_absolute_percentage_error: 11.5704 - val_mean_squared_logarithmic_error: 0.016

In [12]:
model.save('model.h5')


In [ ]:
# del model  # deletes the existing model

## 预测

In [51]:
# calculate predictions

test_X = X
#actual = y

# from keras.models import load_model

# model = load_model('model.h5')
predictions = model.predict(test_X)

In [52]:
# round predictions
B = []
for x in predictions:
    for i in x:
        B.append(i)
        
B = B[1:250000]

In [53]:
y = []
actual = []
with open('../../emotionData/pearson.CSV',"rt", encoding="utf-8") as vsvfile:
    reader = csv.reader(vsvfile)
    y = [row[3] for row in reader]

y = y[2:250001]


actual = y

In [54]:

sum, sum2 = float(0), float(0)
max, mre, res, abres = float(0), float(0), float(0), float(0)

zipped = zip(actual, B)

In [55]:
print("Actual\t\tPredicted\t MRE\t\tRes. Error\tAbs. Res. Error")
for i, j in zipped:
    mre = float(abs(float(i) - float(j))) / float(i)
    res = float(i) - float(j)
    abres = float(abs(res))
    sum = sum + mre
    sum2 = sum2 + res
    if (abs(float(i) - float(j)) / float(i) > max):
        max = float(abs(float(i) - float(j))) / float(i)
    stringI = '{:.4f}'.format(float(i))
    stringJ = '{:.4f}'.format(float(j))
    #print (stringI, "\t\t", stringJ, "\t", '{:.4f}'.format(mre), "\t", '{:.4f}'.format(res), "\t", '{:.4f}'.format(abres))

Actual		Predicted	 MRE		Res. Error	Abs. Res. Error


In [56]:
def mean_squared_error_byself(y_true,y_predict):
    assert len(y_true) == len(y_predict), \
        "the size of y_true must be equal to be the size of y_predict"
    return np.sum((y_true - y_predict) ** 2 ) / len(y_true)

def root_mean_squared_error_byself(y_true,y_predict):
    """计算y_true和y_predict之间的RMES"""
    assert len(y_true) == len(y_predict), \
        "the size of y_true must be equal to be the size of y_predict"
    return math.sqrt(np.sum((y_true - y_predict) ** 2 ) / len(y_true))

def mean_absolute_error_byself(y_true,y_predict):
    """计算y_true和y_predict之间的MAE"""
    assert len(y_true) == len(y_predict), \
        "the size of y_true must be equal to be the size of y_predict"
    return np.sum(np.absolute(y_true - y_predict)) / len(y_true)

print("Mean Absolute Error      ", mean_absolute_error_byself(np.array(actual, dtype = float) , np.array(B, dtype = float)))
print("Mean Squared Error       ", mean_squared_error_byself(np.array(actual, dtype = float) , np.array(B, dtype = float)))
print("Root Mean Square Error : ", root_mean_squared_error_byself(np.array(actual, dtype = float) , np.array(B, dtype = float)))


Mean Absolute Error       0.034898748245977426
Mean Squared Error        0.007486316481229883
Root Mean Square Error :  0.08652350247897898


In [57]:
print("\nMean Magnitude of Relative Error (MMRE) : ", sum / len(actual))
print("MaxMRE :                 ", max)
print("Explained Variance Score ", metrics.explained_variance_score(np.array(actual, dtype = float) , np.array(B, dtype = float)))
print("Mean Absolute Error      ", metrics.mean_absolute_error(np.array(actual, dtype = float) , np.array(B, dtype = float)))
print("Mean Squared Error       ", metrics.mean_squared_error(np.array(actual, dtype = float) , np.array(B, dtype = float)))
print("Median Absolute Error    ", metrics.median_absolute_error(np.array(actual, dtype = float) , np.array(B, dtype = float)))
print("R-Square Score           ", metrics.r2_score(np.array(actual, dtype = float) , np.array(B, dtype = float)))
print("Root Mean Square Error : ", (sum2 / len(B)) ** (1 / 2))


Mean Magnitude of Relative Error (MMRE) :  0.11863252215654474
MaxMRE :                  774.6109140357192
Explained Variance Score  0.8974918908230183
Mean Absolute Error       0.034898748245977426
Mean Squared Error        0.007486316481229883
Median Absolute Error     0.012521769618987988
R-Square Score            0.8960483127125634
Root Mean Square Error :  0.10097624858286167
